<a href="https://colab.research.google.com/github/Manvi190502/AgenticAI/blob/main/RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install langchain langchain-core langchain_community langgraph langchain-huggingface transformers torch langchain_chroma unstructured

In [ ]:
#!pip install unstructured
from logging import LoggerAdapter
from langchain_community.document_loaders import UnstructuredURLLoader

urls=['https://docs.langchain.com/oss/python/langgraph/overview']
Loader = UnstructuredURLLoader(urls=urls)
data = Loader.load()

In [ ]:
data

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
all_splits = text_splitter.split_documents(data)
print("Total number of documents: ",len(all_splits))

In [ ]:
from langchain_community.embeddings import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings()

vector = embeddings.embed_query("hello, world!")
vector[:5]

In [ ]:
from langchain_chroma import Chroma
from langchain_core.documents import Document

vectorstore = Chroma.from_documents(documents=all_splits, embedding=HuggingFaceEmbeddings())

In [ ]:
#if you want to store chromaDb locally
'''
vectorstore = Chroma.from_documents(
    documents=all_splits,
    embedding=HuggingFaceEmbeddings(),
    persist_directory="./chroma_db"    #Custom directory
)
'''

#loading the database later
'''
vectorstore = Chroma(
    persist_directory="./chroma_db",
    embedding_function=HuggingFaceEmbeddings(),
'''

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

model_id = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"

tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
    model_id,
    torch_dtype=torch.float16,
    device_map="auto"
)

text_generation_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=512,
    temperature=0.7,
)

from langchain_huggingface import HuggingFacePipeline
llm = HuggingFacePipeline(pipeline=text_generation_pipeline)


In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_template("""
You are a helpful assistant.
Answer the question using ONLY the provided context.
If the answer is not in the context, say:
"I don't know based on the provided information."

Context:
{context}

Question:
{question}

Answer:
""")


In [ ]:
from typing_extensions import List, TypedDict
from langchain_core.documents import Document

class State(TypedDict):
  question: str
  context: List[Document]
  answer: str

In [ ]:
def retrieve(state: State):
  retrieved_docs = vectorstore.similarity_search(state["question"],k=1)
  return {"context": retrieved_docs}

In [ ]:
def generate(state: State):
  docs_content = "\n\n".join(doc.page_content for doc in state["context"])
  messages = prompt.invoke({"question":state["question"],"context":docs_content})
  response = llm.invoke(messages)
  return {"answer": response}

In [ ]:
from langgraph.graph import START, StateGraph
graph_builder= StateGraph(State).add_sequence([retrieve, generate])
graph_builder.add_edge(START,"retrieve")
graph = graph_builder.compile()

In [ ]:
from IPython.display import display, Image
display(Image(graph.get_graph().draw_mermaid_png()))

In [ ]:
response = graph.invoke({"question":"What is langgraph?"})
print(response["answer"])

Human: 
You are a helpful assistant.
Answer the question using ONLY the provided context.
If the answer is not in the context, say:
"I don't know based on the provided information."

Context:
Skip to main content

Docs by LangChain home page

light logo

dark logo

Deep AgentsLangChainLangGraphIntegrationsLearnReferenceContribute

Overview

Get started

Install

Quickstart

Local server

Changelog

Thinking in LangGraph

Workflows + agents

Capabilities

Persistence

Durable execution

Streaming

Interrupts

Time travel

Memory

Subgraphs

Production

Application structure

Test

LangSmith Studio

Agent Chat UI

LangSmith Deployment

LangSmith Observability

LangGraph APIs

Runtime

Install

Core benefits

LangGraph ecosystem

Acknowledgements

LangGraph overview

Gain control with LangGraph to design agents that reliably handle complex tasks

Question:
What is langgraph?

Answer:
Langgraph is an open-source tool for building and deploying agents that execute code in a distributed and 